## Prediction ON IMDB MOVIE RATING - Sindhura Nadendla

## Load the dataset 

In [25]:
from warnings import filterwarnings
filterwarnings('ignore')

In [26]:
import pandas as pd
df = pd.read_csv('IMDb Movies India.csv',na_values=(' '),encoding='latin-1')
df.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,NaN,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


In [27]:
df.shape

(15509, 10)

In [28]:
df.columns

Index(['Name', 'Year', 'Duration', 'Genre', 'Rating', 'Votes', 'Director',
       'Actor 1', 'Actor 2', 'Actor 3'],
      dtype='object')

## Perform basic data quality checks

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15509 entries, 0 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      15508 non-null  object 
 1   Year      14981 non-null  object 
 2   Duration  7240 non-null   object 
 3   Genre     13632 non-null  object 
 4   Rating    7919 non-null   float64
 5   Votes     7920 non-null   object 
 6   Director  14984 non-null  object 
 7   Actor 1   13892 non-null  object 
 8   Actor 2   13125 non-null  object 
 9   Actor 3   12365 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.2+ MB


In [30]:
df.isna().sum()

Name           1
Year         528
Duration    8269
Genre       1877
Rating      7590
Votes       7589
Director     525
Actor 1     1617
Actor 2     2384
Actor 3     3144
dtype: int64

In [31]:
df.duplicated().sum()

6

There are missing values and duplicated rows in the dataset.

In [32]:
# drop the duplicated rows
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

## Feature Engineering 
Steps performed under this section
1. Dropping statistically insignificant columns

    Movie name doesnt have much significance here. So lets drop that column
    Movie Duration has many missing columns, so lets discard this feature
2. Handling missing values

    Rating, Votes has many missing values, almost half of the rows of dataset are missing. 
    Rating is our target feature, lets handle missing values.
    As per IMDB structure, final rating is determined by aggregated value of votes. Lets handle missing Votes.
3. Convert Year,Votes to int
4. Encoding Categorical features using Target encoding-Mean encoding
5. Separate X and Y features
6. Split the dataset into training data and testing data
-------------------------------------------------------------------------------------------------

1. Dropping statistically insignificant columns

In [33]:
df.drop(columns=['Name','Duration'],axis=1,inplace=True)
df.head()

,Year,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,(2019),Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,(2021),"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,(2019),"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,(2010),Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


In [34]:
df.shape

(15503, 8)

--------------------------------------------------------------------------------
2. Handling missing values

In [35]:
## Drop duplicate rows
df.drop_duplicates(inplace=True)

In [36]:
df.duplicated().sum()

0

In [37]:
df.nunique()

Year         102
Genre        485
Rating        84
Votes       2034
Director    5938
Actor 1     4718
Actor 2     4891
Actor 3     4820
dtype: int64

In [38]:
cat = list(df.columns[df.dtypes=='object'])

In [39]:
mn = df['Rating'].mean()
df['Rating'].fillna(mn,inplace=True)

In [40]:
for i in cat:
    m = df[i].mode()[0]
    df[i].fillna(m,inplace=True)

In [41]:
df.isna().sum()

Year        0
Genre       0
Rating      0
Votes       0
Director    0
Actor 1     0
Actor 2     0
Actor 3     0
dtype: int64

--------------------------------------------------------------------------------
3. Convert Year,Votes to int

In [42]:
# Lets convert Date column from string to int
df['Year']= df['Year'].str.strip('()').astype(int)
# Lets convert Votes column from string to int
df['Votes'] = df['Votes'].str.replace(',','')
df['Votes'] = df['Votes'].str.replace('.','')
df['Votes'] = df['Votes'].str.strip('$')
df['Votes'] = df['Votes'].str.strip('M')
df['Votes'] = df['Votes'].astype(int)

In [43]:
df.nunique()

Year         102
Genre        485
Rating        85
Votes       2034
Director    5938
Actor 1     4718
Actor 2     4891
Actor 3     4820
dtype: int64

--------------------------------------------------------------------------------
4. Encoding Categorical features using Target encoding-Mean encoding

As we can observe from above code that the Below features have high cardinality:

    Votes
    Director
    Actor 1
    Actor 2
    Actor 3

High cardinality : Features having large number of unique values or large number of categories.

If we use One Hot Encoding for such features, it creates large number of binary columns, resulting in memory usage and large computation times. This is not a feasible scenario.

Hence, for this scenario, we will use Target encoding where we calcluate a group aggregation (mean) for each column and subsitute the data with mean values.

I will perform Target Encoding for the above mentioned features
    

In [44]:
## performing mean encoding
df['Genre encoded'] = round(df.groupby('Genre')['Rating'].transform('mean'),1)
df['Votes encoded'] = round(df.groupby('Votes')['Rating'].transform('mean'),1)
df['Director encoded'] = round(df.groupby('Director')['Rating'].transform('mean'),1)
df['Actor 1 encoded'] = round(df.groupby('Actor 1')['Rating'].transform('mean'),1)
df['Actor 2 encoded'] = round(df.groupby('Actor 2')['Rating'].transform('mean'),1)
df['Actor 3 encoded'] = round(df.groupby('Actor 3')['Rating'].transform('mean'),1)

df.drop(columns=['Genre','Votes','Director','Actor 1','Actor 2','Actor 3'],inplace=True)
df['Rating'] = round(df['Rating'],1)
df.head()

,Year,Rating,Genre encoded,Votes encoded,Director encoded,Actor 1 encoded,Actor 2 encoded,Actor 3 encoded
0,2019,5.8,6.0,5.8,5.8,5.8,5.5,5.8
1,2019,7.0,6.0,5.8,7.0,6.8,7.0,7.0
2,2021,5.8,6.3,5.8,5.8,6.2,6.8,5.8
3,2019,4.4,5.7,5.9,4.4,5.4,4.4,4.4
4,2010,5.8,6.0,5.8,6.3,6.8,5.8,5.5


--------------------------------------------------------------------------------
5. Separate X and Y features

In [45]:
X = df.drop(columns='Rating')
Y =df[['Rating']]

In [46]:
X.head()

,Year,Genre encoded,Votes encoded,Director encoded,Actor 1 encoded,Actor 2 encoded,Actor 3 encoded
0,2019,6.0,5.8,5.8,5.8,5.5,5.8
1,2019,6.0,5.8,7.0,6.8,7.0,7.0
2,2021,6.3,5.8,5.8,6.2,6.8,5.8
3,2019,5.7,5.9,4.4,5.4,4.4,4.4
4,2010,6.0,5.8,6.3,6.8,5.8,5.5


In [47]:
Y.head()

,Rating
0,5.8
1,7.0
2,5.8
3,4.4
4,5.8


--------------------------------------------------------------------------------
6. Split the dataset into training data and testing data

In [48]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size=0.25,random_state=21)

In [49]:
xtrain.shape

(11325, 7)

In [50]:
xtest.shape

(3776, 7)

In [51]:
ytrain.shape

(11325, 1)

In [52]:
ytest.shape

(3776, 1)

## Build the model

Performing Algorithm Evaluaton to check which regression models give best results 

In [63]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV

In [54]:
dct = {
    'Linear':LinearRegression(),
    'DecisionTree':DecisionTreeRegressor(),
    'RandomForest':RandomForestRegressor(),
    'GradientBoosting':GradientBoostingRegressor(),
    'KNN':KNeighborsRegressor(),
    'SVR':SVR()
}

In [55]:
dct.items()

dict_items([('Linear', LinearRegression()), ('DecisionTree', DecisionTreeRegressor()), ('RandomForest', RandomForestRegressor()), ('GradientBoosting', GradientBoostingRegressor()), ('KNN', KNeighborsRegressor()), ('SVR', SVR())])

In [56]:
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score

In [57]:
train_mse = []
train_r2 = []
test_mse = []
test_r2 = []
train_cv = []

for name,model in dct.items():
    # fit the model
    m = model.fit(xtrain,ytrain)
    ypred_train = m.predict(xtrain)
    ypred_test = m.predict(xtest)
    # calculate MSE
    mse_train = mean_squared_error(ytrain,ypred_train)
    mse_test = mean_squared_error(ytest,ypred_test)
    # caluclate R2
    r2_train = (r2_score(ytrain,ypred_train))*100
    r2_test = (r2_score(ytest,ypred_test))*100
    # calculate cross validated scores
    cv = cross_val_score(m,xtrain,ytrain,cv=5,scoring='r2')
    scores = (cv.mean())*100

    # add these values to the respective list to compare the output
    train_mse.append(mse_train)
    train_r2.append(r2_train)
    test_mse.append(mse_test)
    test_r2.append(r2_test)
    train_cv.append(scores)

    # print the results
    print(f'Scores for {name}')
    print("Training Scores")
    print(f'MSE:{train_mse}')
    print(f'R2:{train_r2}')
    print("Testing Scores")
    print(f'MSE:{test_mse}')
    print(f'R2:{test_r2}')
    print(f'CV:{train_cv}')
    print("===============================")

Scores for Linear
Training Scores
MSE:[0.24667540151040798]
R2:[74.87422109028986]
Testing Scores
MSE:[0.2715908987362406]
R2:[74.3997681809067]
CV:[74.73950258907246]
Scores for DecisionTree
Training Scores
MSE:[0.24667540151040798, 0.0002136865342163355]
R2:[74.87422109028986, 99.97823438988311]
Testing Scores
MSE:[0.2715908987362406, 0.42744968220338975]
R2:[74.3997681809067, 59.708476954407075]
CV:[74.73950258907246, 58.210539928258264]


In [60]:
res = {'Name':list(dct.keys()),
       'MSE Training Scores':train_mse,
       'MSE Testing Scores':test_mse,
       'R2 Training Scores':train_r2,
       'R2 Testing Scores':test_r2,
       'CV Training Scores':train_cv}

In [62]:
df_res = pd.DataFrame(res)
df_res.sort_values('CV Training Scores',ascending=False)

,Name,MSE Training Scores,MSE Testing Scores,R2 Training Scores,R2 Testing Scores,CV Training Scores
2,RandomForest,0.027219,0.220756,97.227565,79.191516,79.809409
3,GradientBoosting,0.190184,0.237292,80.628352,77.632832,78.409983
0,Linear,0.246675,0.271591,74.874221,74.399768,74.739503
4,KNN,0.181065,0.308287,81.557099,70.940757,71.396368
1,DecisionTree,0.000214,0.427450,99.978234,59.708477,58.210540
5,SVR,0.979322,1.057331,0.248540,0.335717,0.023700


#### Lets consider Random Forest Regressor as its giving good results

In [68]:
params = {'n_estimators':[200,300],
          'max_depth':[5,6,7,8],
          'min_samples_split':[2,3,4,5,6],
          'criterion':['squared_error','absolute_error']}

In [69]:
rfr = RandomForestRegressor()
rscv = RandomizedSearchCV(rfr,params,cv=3,scoring='neg_mean_squared_error')
rscv.fit(xtrain,ytrain)

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(),
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error'],
                                        'max_depth': [5, 6, 7, 8],
                                        'min_samples_split': [2, 3, 4, 5, 6],
                                        'n_estimators': [200, 300]},
                   scoring='neg_mean_squared_error')

In [70]:
rscv.best_params_

{'n_estimators': 200,
 'min_samples_split': 4,
 'max_depth': 8,
 'criterion': 'squared_error'}

In [71]:
best_rfr = rscv.best_estimator_
best_rfr

RandomForestRegressor(max_depth=8, min_samples_split=4, n_estimators=200)

Random Forest model gives score results around 77%. 

I am using XG Boost to check the results and see if this model improves the prediction scores

In [ ]:
from xgboost import XGBRegressor

In [ ]:
model = XGBRegressor()
model.fit(xtrain,ytrain)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
model.score(xtrain,ytrain)

0.9250681221304905

In [ ]:
model.score(xtest,ytest)

0.7841636539213964

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {'n_estimators':[200,300,500,600,800,1000],
          'learning_rate':[0.05,0.1,0.2,0.3],
          'max_depth':[5,6,7,8,9,10],
          'min_child_weight':[1,2,3],
          'objective':['reg:squarederror'],
          'gamma':[0.1,0.2,0.3,0.4]}

In [ ]:
gscv = GridSearchCV(model,params,scoring='neg_mean_squared_error',cv=5)
gscv.fit(xtrain,ytrain)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'gamma': [0.1, 0.2, 0.3, 0.4],
                         'learning_rate': [0.05, 0.1, 0.2, 0.3],
                         'max_depth': [5, 6, 7, 8, 9, 10],
                         'min_child_weight': [1, 2, 3],
                         'n_estimators': [200, 300, 500, 600, 800, 1000],
                         'objective': ['reg:squarederror']},
             scoring='neg_mean_squared_error')

In [ ]:
gscv.best_params_

{'gamma': 0.1,
 'learning_rate': 0.05,
 'max_depth': 5,
 'min_child_weight': 3,
 'n_estimators': 500,
 'objective': 'reg:squarederror'}

In [ ]:
best_xgb = gscv.best_estimator_
best_xgb

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
best_xgb.score(xtrain,ytrain)

0.8574709771358586

In [ ]:
best_xgb.score(xtest,ytest)

0.8021907473809164

Lets check by tuning other parameters to this model

In [ ]:
params1 = {'subsample':[0.5,0.6,0.7,0.8,0.9,1],
           'colsample_bytree':[0.5,0.6,0.7,0.8,0.9,1]}

In [ ]:
gscv1 = GridSearchCV(best_xgb,params1,cv=5,scoring='neg_mean_squared_error')
gscv1.fit(xtrain,ytrain)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=0.1,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=0.05, ma...
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=5, max_leaves=None,
                                    min_child_weight=3, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=500,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1],
                         'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1]},
             scoring='neg_mean_squared_error')

In [ ]:
gscv1.best_params_

{'colsample_bytree': 0.8, 'subsample': 0.9}

In [ ]:
best_xgb2 = gscv1.best_estimator_
best_xgb2

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
best_xgb2.score(xtrain,ytrain)

0.8818121398218832

In [ ]:
best_xgb2.score(xtest,ytest)

0.8047686940495138

## Evaluate the models : Random Forest and XGBoost

In [76]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [79]:
def eval_model(model,xtrain,ytrain):
    # Fit the model
    model.fit(xtrain,ytrain)
    # Predict the scores
    ypred_test = model.predict(xtest)
    # Calculate MSE,RMSE,MAE,R2 scores
    mse = mean_squared_error(ytest,ypred_test)
    rmse = mse**(1/2)
    mae = mean_absolute_error(ytest,ypred_test)
    r2 = r2_score(ytest,ypred_test)
    return mse,rmse,mae,r2

Random Forest Evaluation Metrics

In [80]:
(MSE,RMSE,MAE,r2) = eval_model(best_rfr,xtrain,ytrain)
print(f'Evaluation Metrics: \nMSE: {MSE}\nRMSE:{RMSE}\nMAE:{MAE}\nR2:{r2}')

Evaluation Metrics: 
MSE: 0.23569525214078613
RMSE:0.48548455396725665
MAE:0.2674453628263474
R2:0.7778330157033859


XGBoost Evaluation Metrics

In [ ]:
(MSE,RMSE,MAE,r2) = eval_model(best_xgb2,xtrain,ytrain)
print(f'Evaluation Metrics: \nMSE: {MSE}\nRMSE:{RMSE}\nMAE:{MAE}\nR2:{r2}')

Evaluation Metrics: 
MSE: 0.20711939727435047
RMSE:0.45510372144638683
MAE:0.25907305710143963
R2:0.8047686940495138


#### XGBoost is providing best test score as compared to other models. Considering XGBoost for final prediction
--------------------------------------------------------------------------------------

## Model Prediction

In [ ]:
ypred_test = best_xgb2.predict(xtest)
ypred_test[:10]

array([6.9940157, 5.766688 , 4.120275 , 7.9405217, 5.923771 , 6.8461776,
       6.0055513, 5.437491 , 5.8292665, 4.838405 ], dtype=float32)

In [ ]:
ytest.head(10)

,Rating
4806,7.1
10573,5.8
3674,3.7
9179,8.0
6229,7.3
13993,6.9
7125,6.2
2874,5.8
7049,5.8
6739,4.9


In [ ]:
df_final = xtest
df_final['Predicted_Rating'] = ypred_test
df_final

,Year,Genre encoded,Votes encoded,Director encoded,Actor 1 encoded,Actor 2 encoded,Actor 3 encoded,Predicted_Rating
4806,2015,5.4,7.1,6.4,5.5,6.2,5.7,6.994016
10573,1989,5.7,5.8,5.8,5.6,5.8,5.8,5.766688
3674,2000,5.9,3.7,4.9,5.2,5.8,5.8,4.120275
9179,1970,5.9,8.0,7.1,6.8,6.8,6.8,7.940522
6229,2003,5.7,6.1,5.7,5.1,5.6,6.9,5.923771
...,...,...,...,...,...,...,...,...
5126,2007,5.8,5.7,5.6,5.7,5.8,5.6,5.548202
5934,1971,5.8,6.2,6.0,6.4,6.0,6.6,6.665455
15146,1972,5.3,5.7,5.8,5.3,6.1,6.0,5.353926
14280,2010,6.0,6.2,5.7,5.1,5.7,5.7,5.727316


Save the results to dataframe

In [ ]:
df_final.to_csv('Predicted Ratings.csv',index=False)